In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import os
import ast

In [2]:
movies = pd.read_csv('/Users/arpit_lal/Desktop/tmdb_5000_movies.csv')
credits = pd.read_csv('/Users/arpit_lal/Desktop/tmdb_5000_credits.csv')
# rating = pd.read_csv("/Users/arpit_lal/Desktop/ratings.csv",low_memory=False)
# keywords = pd.read_csv("/Users/arpit_lal/Desktop/keywords.csv",low_memory=False)

In [3]:
print(type(movies))
print(type(credits))


<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


In [4]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,production_countries,release_date,revenue,runtime,spoken_languages,status,tagline,title,vote_average,vote_count
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...","[{""iso_3166_1"": ""US"", ""name"": ""United States o...",2009-12-10,2787965087,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800


In [5]:
credits.head(1)

,movie_id,title,cast,crew
0,19995,Avatar,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [6]:
movies = movies.merge(credits,on='title')

In [7]:
print(type(movies))


<class 'pandas.core.frame.DataFrame'>


In [8]:
movies.head(1)

,budget,genres,homepage,id,keywords,original_language,original_title,overview,popularity,production_companies,...,runtime,spoken_languages,status,tagline,title,vote_average,vote_count,movie_id,cast,crew
0,237000000,"[{""id"": 28, ""name"": ""Action""}, {""id"": 12, ""nam...",http://www.avatarmovie.com/,19995,"[{""id"": 1463, ""name"": ""culture clash""}, {""id"":...",en,Avatar,"In the 22nd century, a paraplegic Marine is di...",150.437577,"[{""name"": ""Ingenious Film Partners"", ""id"": 289...",...,162.0,"[{""iso_639_1"": ""en"", ""name"": ""English""}, {""iso...",Released,Enter the World of Pandora.,Avatar,7.2,11800,19995,"[{""cast_id"": 242, ""character"": ""Jake Sully"", ""...","[{""credit_id"": ""52fe48009251416c750aca23"", ""de..."


In [9]:
movies = movies[['movie_id','title','overview','genres','keywords','cast','crew',]] 

In [96]:
movies.isnull().sum()

movie_id    0
title       0
overview    0
genres      0
keywords    0
cast        0
crew        0
tags        0
dtype: int64

In [94]:
movies.dropna(inplace=True)

In [98]:
movies.duplicated().sum()

TypeError: unhashable type: 'list'

In [ ]:
movies.head(2)

In [ ]:
movies.iloc[0]['genres']

In [ ]:
def convert(obj):
    l = []
    for i in ast.literal_eval(obj):
        l.append(i['name'])
    return l

In [ ]:
movies['keywords'] = movies['keywords'].apply(convert)
movies['genres'] = movies['genres'].apply(convert)


In [ ]:
def convert3(obj):
    l = []
    counter = 0
    for i in ast.literal_eval(obj):
        if counter != 3:
            l.append(i['name'])
            counter+=1
        else:
            break
    return l


In [ ]:
movies['cast'] = movies['cast'].apply(convert3)

In [ ]:
movies.head()

In [ ]:
def fectch_director(obj):
    l = []
    for i in ast.literal_eval(obj):
        if i['job'] == 'Director':
            l.append(i['name'])
            break
    return l

In [ ]:
movies['crew'] = movies['crew'].apply(fectch_director)

In [ ]:
movies.head()

In [ ]:
movies['overview'] = movies['overview'].apply(lambda x: x.split() if isinstance(x, str) else x)

In [ ]:
movies.head()

In [ ]:
movies['genres'] = movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords'] = movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast'] = movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew'] = movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tags'] = movies['genres'] + movies['keywords'] + movies['cast'] + movies['crew']

In [ ]:
new_df = movies[['movie_id','title','tags']]

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:" ".join(x))

In [ ]:
# # Drop rows with NaN values
# # Truncate text
# # new_df['tags'] = new_df['tags'].str[:100]  # Keep first 100 characters
# # Subset by random rows
# new_df = new_df.sample(n=10000)  # Keep 1000 row
# #new_df = new_df.sample(n=50000, replace=True)

# print(new_df.info(memory_usage='deep'))

In [ ]:
new_df['tags'] = new_df['tags'].apply(lambda x:x.lower())
new_df.head() 

In [ ]:
import nltk       #stem used for combinning similar kind of words or words holding same meaning, so that we could compare and recommend
from nltk.stem import PorterStemmer

In [ ]:
ps = PorterStemmer()

In [ ]:
def stems(text):
    l = []
    for i in text.split():
        l.append(ps.stem(i))

    return " ".join(l)

In [ ]:
new_df['tags'] = new_df['tags'].apply(stems)

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
cv = CountVectorizer(max_features=5000, stop_words= 'english')

In [ ]:
vector = cv.fit_transform(new_df['tags']).toarray()

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity 

In [ ]:
similarity = cosine_similarity(vector)

In [ ]:
similarity.shape

In [ ]:
similarity[0]

In [ ]:
sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

In [ ]:
def recommend(movie):
    movie_index = new_df[new_df['title'] == movie].index[0]
    distances = similarity[movie_index]
    movies_list = sorted(list(enumerate(similarity[0])),reverse=True,key=lambda x:x[1])[1:6]

    for i in movies_list:
        print(new_df.iloc[i[0]].title)

In [ ]:
recommend('The Dark Knight Rises')

In [ ]:
import os
import pickle

# Save files in 'artifacts' folder within the project directory
# os.makedirs('artifacts', exist_ok=True)  # Create folder if it doesn't exist
pickle.dump(new_df.to_dict(),open('movies_dict.pkl','wb'))
pickle.dump(similarity,open('similarity.pkl', 'wb'))


In [ ]:
new_df.to_dict()